#Imports

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Widgets

In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.text("catalog","catalog_ecommerce")
dbutils.widgets.text("schema_source", "bronze")
dbutils.widgets.text("schema_sink", "silver")

#Constantes

In [0]:
catalog         =  dbutils.widgets.get("catalog")
schema_source   =  dbutils.widgets.get("schema_source")
schema_sink     = dbutils.widgets.get("schema_sink")

#Preparacion

In [0]:
df_items = spark.table(f"{catalog}.{schema_source}.orders_items")

#Limpieza

In [0]:
df_items_clean = df_items.filter(col("order_id").isNotNull())\
    .filter(col("product_id").isNotNull())\
    .filter(col("seller_id").isNotNull())

#Conversion

In [0]:
df_items_clean = df_items_clean.withColumn("Total_pay", col("price")+col("freight_value"))\
    .withColumn("has_freight" , when(col("freight_value")>0,"Y").otherwise("N"))

In [0]:
df_items_clean.write.mode("overwrite").saveAsTable(f"{catalog}.{schema_sink}.orders_items")